In [1]:
import pandas as pd
import requests 
import os
import xml.etree.ElementTree as ET
import gzip




In [2]:
def unzip():
    with gzip.open('incident.xml.gz', 'rb') as f_in:
        with open('incident.xml', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)



In [8]:
def incidents():
    incident_dirs = []
    roads = []
    locations = []
    names = []
    events = []
    parsedXML = ET.parse( "incident.xml" )
    root = parsedXML.getroot()
    for child in root:
        names.append(child.attrib['name'])
        incident_dirs.append(child.attrib['dir'])
        roads.append(child.attrib['road'])
        locations.append(child.attrib['location'])
        events.append(child.attrib['event_type'])
 

    DF = pd.DataFrame({"name" : names,
                       "direction" :incident_dirs,
                       "Road" : roads,
                       "Location" : locations,
                       "Event" : events

    })
    print(DF)






In [9]:
incidents()

               name direction     Road         Location              Event
0  2018092819553371        WB   T.H.62        @ T.H.100  INCIDENT_ROADWORK
1  2018092819531862        SB  U.S.169      @ T.H.62 WB  INCIDENT_ROADWORK
2  2018092819533003        NB  U.S.169      @ T.H.62 WB  INCIDENT_ROADWORK
3      L004_9939905        NB    I-35W     @ T.H.121 NB     INCIDENT_CRASH
4  2018092819315453        NB  T.H.100      @ T.H.62 WB  INCIDENT_ROADWORK
5  2018092820101610        SB    I-35W  @ Cleveland Ave  INCIDENT_ROADWORK
6      L004_9940001        NB  T.H.100       @ I-394 WB     INCIDENT_STALL
7  2018092820040732        SB    I-35E       S of I-35W  INCIDENT_ROADWORK
8  2018092820090159        WB   T.H.36  @ Cleveland Ave  INCIDENT_ROADWORK
9  2018092819042630        SB  T.H.100      @ T.H.62 WB  INCIDENT_ROADWORK


NameError: name 'DF' is not defined